In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
import pickle

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

In [5]:
data = pd.read_csv("Churn_Modelling.csv")

data.head(10)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2,134603.88,1,1,1,71725.73,0


In [8]:
# Drop PII (Irelavant Feature)

data.drop(['RowNumber','CustomerId','Surname'],axis=1,inplace=True)

# Encode Categorical Data

label_gender = LabelEncoder()
data['Gender'] = label_gender.fit_transform(data['Gender'])

# One hot Encode - Geography Column - if Geography column uses Label Encoder, then there are chances that Germany is assigned to 2, spain with 1.
# In an ANN model, this means data with higher value takes precedence which should not happen in our case

encoder_geography=OneHotEncoder()
encoded_values = encoder_geography.fit_transform(data[['Geography']])

encoded_df = pd.DataFrame(encoded_values.toarray(),columns=encoder_geography.get_feature_names_out(['Geography']))

# Combine one Hot Encoded data with the original data

data = pd.concat([data.drop(['Geography'],axis=1),encoded_df],axis=1)

# Train - test Split Data with Dependent and Independent Features

x= data.drop(['Exited'],axis=1)
y = data['Exited']

x_train,x_test,y_train,y_test = train_test_split(x,y,random_state=42, test_size= 0.2)

# Scaling the Features

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [9]:
# Define a function to create the model and try different parameters(Keras Classifier)

def create_model(neurons = 32, layers = 1):
    model=Sequential()
    # Input Layers
    model.add(Dense(
    neurons,
    activation = 'relu',
    input_shape = (x_train.shape[1],)
    )
    )

    #Number of Hidden layers depend on the input value been sent
    # Hidden Layer
    
    for _ in range(layers-1):
        model.add(
            Dense(
                neurons,
                activation='relu'
        )
        )

    # Output Layer

    model.add(
        Dense(
            1,
            activation='sigmoid'
        )
    )

    # Compile the model

    model.compile(
        optimizer='adam',
        loss ='binary_crossentropy',
        metrics=['accuracy']
    )

    return model

In [14]:
# Create a Keras Classifier

model= KerasClassifier(
    build_fn=create_model,
    layers =1,
    neurons=32,
    epochs=50,
    batch_size=10,
    verbose=1
)

In [16]:
# Define the Grid Search CV Parameters

param_grid={
    'neurons':[16,32,64,128],
    'layers':[1,2],
    'epochs':[50,100]
}

grid = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    n_jobs= -1,
    cv=3
)

grid_result = grid.fit(x_train,y_train)

# Print the best score and parameters

print(f"{grid_result.best_score_},{grid_result.best_params_}")

c:\Users\Swetha\OneDrive\Desktop\Python & ML Related\Deep Learning\DL - Udemy\ANN\venv\lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\Swetha\OneDrive\Desktop\Python & ML Related\Deep Learning\DL - Udemy\ANN\venv\lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8074 - loss: 0.4422
Epoch 2/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8430 - loss: 0.3847
Epoch 3/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8530 - loss: 0.3605
Epoch 4/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8574 - loss: 0.3510
Epoch 5/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8537 - loss: 0.3469
Epoch 6/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8579 - loss: 0.3428
Epoch 7/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8591 - loss: 0.3422
Epoch 8/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8580 - loss: 0.3406
Epoch 9/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8606 - loss: 0.3394
Epoch 10/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8597 - loss: 0.3381
Epoch 11/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8606 - loss: 0.3362
Epoch 12/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step